In [1]:
!pip install PyPDF2

In [3]:
from PyPDF2 import PdfFileReader
from nltk.tokenize import SpaceTokenizer 
import pandas as pd
import os
import re
import numpy as np

In [14]:
root_path = './documents'
pdf_files = []

# get all pdf files
for dir_path, dir_names, filenames in os.walk(root_path):
    for filename in [f for f in filenames if f.endswith(".pdf")]:
        pdf_files.append(os.path.join(dir_path, filename))

In [23]:
text_data = [] 

# get pdf content of all documents
for file in pdf_files:
    pdf_file = PdfFileReader(file)
    pdf_text = ''
    for page in range(pdf_file.numPages):
        pdf_text += pdf_file.getPage(page).extractText()
    
    pdf_text = pdf_text.replace('\n', '').replace('\\', '')
    text_data.append(pdf_text.lower())

In [24]:
print(len(text_data))
print(len(pdf_files))
text_data[0]

28
28


'estado do rio grande do norte câmara municipal do natal  palácio padre miguelinho    1     mesa diretora  ordem do dia: 25/04/2019  presidente:            1º secretário:           2º secretário:             1º orador - ver. dagô do forró  dem - 14h/14h15 2º orador  ver. dickson nasser júnior  psdb- 14h15/14h30 3º orador - ver. dinarte torres - pmb - 14h30/14h45 4º orador  ver.a divaneide basílio - pt - 14h45/15h 5º orador - ver.a eleika bezerra   psl - 15h/15h15 6º orador  ver. eriko jácome  pode - 15h15/15h30                                                       em pauta:  projeto de lei n.º 136/2014  ver. aroldo alves  psdb, subscrito pelo ver. preto aquino  patriota. assunto:                                     - autoriza a compensação de dívidas de qualquer natureza perante o município de natal com créditos líquidos certos, vencidos ou vincendos, ainda de que natureza tributária, de titularidade de servidores públicos municipais e dá outras providências movimento:                 

In [6]:
template_dic_keys = ['n requerimento', 'vereador', 'partido', 'assunto']

In [7]:
# Achei que iria ficar legal categorizar a informação do texto bruto em uma lista de dicionarios.
# Acredito que está funcionando.

list_of_dict = []

for pdf_text in text_data:
    
    text_vec = SpaceTokenizer().tokenize(pdf_text)
    it = 0
    
    while it != len(text_vec)-1:
        temp_dic = dict.fromkeys(template_dic_keys)   
        if text_vec[it] == 'requerimento' and text_vec[it+1] == 'nº.':
            temp_dic['n requerimento'] = text_vec[it+2]
            it = it + 4
            ver_name = ''
            while text_vec[it] != '-':
                ver_name += text_vec[it]
                ver_name += ' '
                it += 1
            temp_dic['vereador'] = ver_name
            temp_dic['partido'] = text_vec[it+1]
            it+=8
            subject = ''
            while text_vec[it] != 'movimento:':
                subject += text_vec[it]
                subject += ' '
                it += 1
            temp_dic['assunto'] = subject
        it += 1
        if temp_dic['assunto']:
            list_of_dict.append(temp_dic)


In [8]:
list_of_dict[0]

{'n requerimento': '01119/2019',
 'vereador': 'ver.a ana paula ',
 'partido': 'psdc',
 'assunto': '- requer que seja enviado ofício à semsur, solicitando a troca de lâmpada queimada no poste nº g7 330, situado na rua alterosa - conjunto pirangi i etapa - neópolis '}

In [9]:
list_of_dict[-1]

{'n requerimento': '00496/2019',
 'vereador': 'ver.  sueldo medeiros  phs   assunto:       ',
 'partido': 'requer',
 'assunto': 'solicitando que o município do natal regulamente, via decreto, a lei federal no 12.846/13. '}

In [10]:
pd.DataFrame.from_dict(list_of_dict).to_csv('requerimentos_data.csv')